In [65]:
import hls4ml
print("hls4ml version:", hls4ml.__version__)


hls4ml version: 0.9.0.dev5+g033d4382


In [1]:
import numpy as np
import pandas as pd
import keras
from keras import layers
import tensorflow as tf
import qkeras
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
#from callbacks import all_callbacks

In [87]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to [0, 1]
x_train = np.round(x_train.astype("float32") / 255).astype("int")
x_test = np.round(x_test.astype("float32") / 255).astype("int")
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [89]:
trim = 4
x_train_ = x_train[:, trim:28-trim, trim:28-trim].reshape(x_train.shape[0],-1)
x_test_ = x_test[:, trim:28-trim, trim:28-trim].reshape(x_test.shape[0],-1)
input_shape = x_train_.shape[1:]
print(input_shape)


(400,)


In [90]:
x_train_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

The following

In [91]:
id_ = 100
tmp = "".join(x_test_[id_].astype("str"))[::-1]
#print(f"b{tmp}")
print(f"b{tmp}")
hex(int(tmp,2)), np.argmax(y_test[id_])#, np.argmax(model.predict(x_train_[id_].reshape(1,-1)))

b0000000000000000000000000000000000000000000000111110000000000000111001111000000000011000000111000000001000011100110000000100000001000110000001000000010001100000010000000100001000000100000001000010000001000000110000100000010000011000001000000011111100000010000000011100000001100000000000000000011000000000000000000100000000000000000001000000000000000000110000000000000000001000000000000000000110000000


('0x3e000e780181c021cc04046040460404204042040c20418203f0201c060000600004000040000c00008000180',
 6)

In [76]:
clipboard = ""
for i,j in enumerate(range(0,400,28)):
    print(f"{i}    {bin(i)[2:].zfill(4)}{tmp[j:j+28][::-1].zfill(28)[::-1]}")

0    00000000001111111100000000000111
1    00011111111000000000111000000110
2    00100000000111000000011100000011
3    00111000000000110000001100000000
4    01000001100001100000000000011000
5    01010110000000000001100001000000
6    01100000001110000100000000000011
7    01111000010000000000001100000100
8    10000000000001110000011000000000
9    10011111000001100000000011100000
10    10100011000000011100000000011001
11    10111011110000000001110111111000
12    11000000000011111110000000000000
13    11010100110000000000000000000000
14    11100000000000000000000000000000


In [92]:
import subprocess
clipboard = ""
for i,j in enumerate(range(0,400,28)):
    clipboard += f"DATA_IN = 32'b{bin(i)[2:].zfill(4)}{tmp[j:j+28][::-1].zfill(28)[::-1]};\n"
    clipboard += "\t\t#dly;"
subprocess.run("clip", text=True, input=clipboard)

CompletedProcess(args='clip', returncode=0)

In [78]:
x_test.shape

(10000, 28, 28, 1)

In [46]:
tmp2 = x_test_[id_].reshape(20,20)
for i in range(len(tmp2)):
    print("".join(np.round(tmp2[i]).astype("int").astype("str")))

00000000000000000000
00000000000000000000
00000000000000000000
00011100000000000000
00111111111111111000
00000011111111111100
00000000000000011000
00000000000000011000
00000000000000110000
00000000000001110000
00000000000001100000
00000000000011100000
00000000000011000000
00000000000011000000
00000000000110000000
00000000001110000000
00000000011100000000
00000000011000000000
00000000110000000000
00000001110000000000


In [45]:
tmp2 = x_test[id_].squeeze()
for i in range(len(tmp2)):
    print("".join(np.round(tmp2[i]).astype("int").astype("str")))

0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000111000000000000000000
0000001111111111111110000000
0000000000111111111111000000
0000000000000000000110000000
0000000000000000000110000000
0000000000000000001100000000
0000000000000000011100000000
0000000000000000011000000000
0000000000000000111000000000
0000000000000000110000000000
0000000000000000110000000000
0000000000000001100000000000
0000000000000011100000000000
0000000000000111000000000000
0000000000000110000000000000
0000000000001100000000000000
0000000000011100000000000000
0000000000011100000000000000
0000000000011100000000000000
0000000000011000000000000000
0000000000000000000000000000


In [16]:
x_train_.shape, x_test_.shape,

((60000, 400), (10000, 400), (60000, 10))

In [68]:
model = Sequential()
model.add(keras.Input(shape=input_shape, name="input0"))


model.add(
    QDense(
        10,
        name='fc1',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu1'))
model.add(
    QDense(
        10,
        name='fc2',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu2'))
model.add(
    QDense(
        num_classes,
        name='fc3',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(Activation(activation='softmax', name='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (QDense)                (None, 10)                4010      
                                                                 
 relu1 (QActivation)         (None, 10)                0         
                                                                 
 fc2 (QDense)                (None, 10)                110       
                                                                 
 relu2 (QActivation)         (None, 10)                0         
                                                                 
 fc3 (QDense)                (None, 10)                110       
                                                                 
 softmax (Activation)        (None, 10)                0         
                                                                 
Total params: 4230 (16.52 KB)
Trainable params: 4230 (16

In [71]:
batch_size = 128
epochs = 15

model.compile(optimizer="adam", loss=['categorical_crossentropy'], metrics=['accuracy'])
model.fit(x_train_, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.9031 - val_loss: 0.4246 - val_accuracy: 0.9127
Epoch 2/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4491 - accuracy: 0.9046 - val_loss: 0.4204 - val_accuracy: 0.9142
Epoch 3/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4470 - accuracy: 0.9055 - val_loss: 0.4192 - val_accuracy: 0.9158
Epoch 4/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4458 - accuracy: 0.9063 - val_loss: 0.4230 - val_accuracy: 0.9122
Epoch 5/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4449 - accuracy: 0.9062 - val_loss: 0.4256 - val_accuracy: 0.9123
Epoch 6/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4456 - accuracy: 0.9055 - val_loss: 0.4261 - val_accuracy: 0.9118
Epoch 7/15
422/422 [==============================] - 0s 1ms/step - loss: 0.4440 - accuracy: 0.9073 - val_loss: 0.4224 - val_accuracy: 0.9160
Epoch 

In [72]:
score = model.evaluate(x_test_, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

save_model(model, "qModel.keras")

Test loss: 0.45980751514434814
Test accuracy: 0.9036999940872192


In [37]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
config['LayerName']['input0']['Precision'] = {'result': 'ap_uint<1>'}  # Set input precision to 1-bit
print("-----------------------------------")
print(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_1/hls4ml_prj',
    part='xcu250-figd2104-2L-e', backend='Vitis'
)
hls_model.compile()

Interpreting Sequential
Topology:
Layer name: input0, layer type: InputLayer, input shapes: [[None, 400]], output shape: [None, 400]
Layer name: fc1, layer type: QDense, input shapes: [[None, 400]], output shape: [None, 10]
Layer name: relu1, layer type: Activation, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: fc2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: relu2, layer type: Activation, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: fc3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: softmax, layer type: Softmax, input shapes: [[None, 10]], output shape: [None, 10]
-----------------------------------
{'Model': {'Precision': 'fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency', 'BramFactor': 1000000000, 'TraceOutput': False}, 'LayerName': {'input0': {'Trace': False, 'Precision': {'result': 'ap_uint<1>'}}, 'fc1': {'Trace': False, 'Precision': {'result': 'fixed<16,6>', 

FileNotFoundError: Could not find module 'C:\Users\Tim\PycharmProjects\rs_final\model_1\hls4ml_prj\firmware\myproject-eC7Bc7Fa.so' (or one of its dependencies). Try using the full path with constructor syntax.

In [38]:
hls_model.write()

Writing HLS project
Done


In [41]:
import os
os.environ['PATH'] += os.pathsep + 'D:/ProgramFiles/Xilinx/Vitis_HLS/2023.2/bin'
hls_model.build(csim=0)

Vivado synthesis report not found.
Cosim report not found.
Timing report not found.


{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '4.344',
  'BestLatency': '10',
  'WorstLatency': '10',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'BRAM_18K': '12',
  'DSP': '10',
  'FF': '2629',
  'LUT': '61606',
  'URAM': '0',
  'AvailableBRAM_18K': '5376',
  'AvailableDSP': '12288',
  'AvailableFF': '3456000',
  'AvailableLUT': '1728000',
  'AvailableURAM': '1280'}}

python3: can't open file 'hls4ml': [Errno 2] No such file or directory
